In [5]:
import cv2
import numpy as np
import random
import mediapipe as mp
import pyautogui 
import math
import screen_brightness_control as sbc
import util
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pynput.mouse import Button, Controller
mouse = Controller()

In [6]:
# Initialize volume control (global)
try:
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))#It fetches the system's speaker device.
    vol_range = volume.GetVolumeRange()   #It activates the volume control interface.
    min_vol, max_vol = vol_range[0], vol_range[1] #t retrieves the minimum and maximum volume levels supported by the device (in decibels).


except Exception as e:
    print(f"Volume control disabled: {e}")
    volume = None

In [7]:
screen_width, screen_height = pyautogui.size()
mouse = Controller()
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode = False,
    model_complexity = 1,
    min_detection_confidence = 0.7,
    min_tracking_confidence = 0.7,
    max_num_hands = 1
)

In [8]:
def find_finger_tip (processed):
    if processed.multi_hand_landmarks: # # Check if any hand landmarks are detected
   
        hand_landmarks  = processed.multi_hand_landmarks[0] #Get the landmarks of the first detected hand (assuming one hand for now)
      
        return hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]#Return the coordinates of the index finger tip
      
    return None

In [9]:
def move_mouse( index_finger_tip):
    if index_finger_tip is not None: #Check if the index finger tip position is available
        #Convert the normalized x and y coordinates (0 to 1) to screen pixel coordinates
        x = int(index_finger_tip.x*screen_width)
        y = int(index_finger_tip.y*screen_height)
        # Move the mouse pointer to the calculated (x, y) screen coordinates
      
        pyautogui.moveTo(x, y)

In [10]:
def is_left_click(landmark_list, thumb_index_dist):
    return (
            util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and
            util.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) > 90 and
            thumb_index_dist > 50  # Thumb and index are apart
    )
def is_right_click(landmark_list, thumb_index_dist):
    
    return (
            util.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50 and
            util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) > 90 and
            thumb_index_dist > 50 # Thumb and index are apart
    )

def is_double_click(landmark_list, thumb_index_dist):
    return (
            util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and
            util.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50 and
            thumb_index_dist > 50 # Thumb and index not touching
    )

def is_screenshot(landmark_list, thumb_index_dist):
    return (
            util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and
            util.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50 and
            thumb_index_dist < 50 # Thumb is close to index → pinch gesture
    )

# def is_volume_control(landmark_list, thumb_index_dist):
#     return (
#         util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) > 80 and  # index finger open
#         util.get_angle(landmark_list[2], landmark_list[3], landmark_list[4]) > 80 and  # thumb open
#         thumb_index_dist > 40  # enough distance
#     )
# def is_brightness_control(landmark_list,thumb_index_dist):
#     return (
#         util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) > 80 and   # index open
#         util.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) > 80     # middle open
#     )

        
        

In [11]:
def detect_gestures(frame, landmark_list, processed):
    if len(landmark_list) >= 21:

        index_finger_tip = find_finger_tip (processed)
        thumb_index_dist = util.get_distance([landmark_list[4], landmark_list[5]])
        if  thumb_index_dist < 50 and util.get_angle(landmark_list[5],landmark_list[6],landmark_list[8]) > 90:
            move_mouse( index_finger_tip)

         #for left click
        elif is_left_click(landmark_list, thumb_index_dist):
            mouse.press(Button.left)
            mouse.release(Button.left)
            cv2.putText(frame, "Left click",(50,50), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,255,0),2)
            
            
        #for right click
        elif is_right_click(landmark_list, thumb_index_dist):
            mouse.press(Button.right)
            mouse.release(Button.right)
            cv2.putText(frame, "Right click",(50,50), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),2)
            
            
        #for double click
        elif is_double_click(landmark_list, thumb_index_dist):
            pyautogui.doubleClick()
            cv2.putText(frame, "Double click",(50,50), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,0),2)
            

        
         #for screenshot
        elif is_screenshot(landmark_list,thumb_index_dist):
           
            im1 = pyautogui.screenshot()
            label = random.randint(1,1000)
            im1.save(f"my_screenshot_{label}.png")
            cv2.putText(frame, "screenshot taken",(50,50), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,0),2)

     

            
            
            

          
                                               
       

In [16]:
def update_volume(thumb, index):
    global volume
    
    if volume is None:
        return
        
    try:
        # Calculate distance between thumb and index (0.0 to ~0.3 typically)
        vol_dist = math.hypot(index[0] - thumb[0], index[1] - thumb[1])
        
        # Clamp and interpolate to 0.0-1.0 range
        vol_dist = max(0.02, min(0.15, vol_dist))  # Constrain to working range
        vol_level = np.interp(vol_dist, [0.02, 0.15], [0.0, 1.0])  # Must be 0.0-1.0
        
        # Validate before setting
        if 0.0 <= vol_level <= 1.0:
            volume.SetMasterVolumeLevelScalar(vol_level, None)
            print(f"Volume set to {int(vol_level*100)}%")  # Debug
        else:
            print(f"Invalid volume level: {vol_level}")
            
    except Exception as e:
        print(f"Volume error: {e}")
        # Attempt to reinitialize if interface is bad
        # global volume
        volume = init_audio()  # You'll need this function

def update_brightness(index, middle):
    try:
        bright_dist = math.hypot(index[0] - middle[0], index[1] - middle[1])
        bright_level = np.interp(bright_dist, [0.02, 0.15], [0, 100])
        sbc.set_brightness(int(bright_level))
    except Exception as e:
        print(f"Brightness error: {e}")

In [17]:

# def main():
    # cap = cv2.VideoCapture(0)  # Fix: "VideoCapture" (correct spelling), 0 for default camera
    # mp_drawing = mp.solutions.drawing_utils  # Fix: Correct module path
    # mp_hands = mp.solutions.hands

    # hands = mp_hands.Hands()  # Initialize MediaPipe Hands

    # try:
    #     while cap.isOpened():  # Fix: "isOpened" (correct capitalization)
    #         ret, frame = cap.read()

    #         if not ret:  # If frame is not read correctly, break
    #             break

    #         frame = cv2.flip(frame, 1)  # Mirror the frame
    #         frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Fix: "cvtColor" (correct spelling)
    #         processed = hands.process(frameRGB)

    #         landmark_list = []

    #         if processed.multi_hand_landmarks:
    #             hand_landmarks = processed.multi_hand_landmarks[0]
    #             mp_drawing.draw_landmarks(  # Fix: Use mp_drawing
    #                 frame, 
    #                 hand_landmarks, 
    #                 mp_hands.HAND_CONNECTIONS  # Fix: "HAND_CONNECTIONS" (correct spelling)
    #             )

#                 for lm in hand_landmarks.landmark:  # Fix: "lm" (not "in" or "ln")
#                     landmark_list.append((lm.x, lm.y))
#                 print(landmark_list)

#             detect_gestures(frame, landmark_list, processed)  # Fix: "detect_gestures" (correct spelling)

#             cv2.imshow("Frame", frame)

#             if cv2.waitKey(1) & 0xFF == ord('q'):  # Fix: "0xFF" (not "DXFF"), colon (not semicolon)
#                 break

#     finally:
#         cap.release()
#         cv2.destroyAllWindows()

# if __name__ == "__main__":
#     main()

def main():
    # Initialize camera (corrected spelling)
    cap = cv2.VideoCapture(0)  # Fix: "VideoCapture" 
    if not cap.isOpened():
        print("Error: Camera not accessible")
        return

    # MediaPipe setup (corrected paths)
    mp_drawing = mp.solutions.drawing_utils  # Fix: "drawing_utils"
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )

    # --- Throttle variables ---
    frame_counter = 0
    update_interval = 5  # Update every 5 frames

    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Mirror and convert color
            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Fix: "cvtColor"
            processed = hands.process(frame_rgb)

            if processed.multi_hand_landmarks:
                hand_landmarks = processed.multi_hand_landmarks[0]
                
                # Draw landmarks (corrected constant)
                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS  # Fix: "HAND_CONNECTIONS"
                )

                # Extract landmarks (fixed variable name)
                landmark_list = [(lm.x, lm.y) for lm in hand_landmarks.landmark]  # Fix: "lm"

                # --- Step 4: Volume/Brightness Control ---
                if frame_counter % update_interval == 0 and len(landmark_list) >= 13:
                    # Volume (thumb [4] vs index [8])
                    update_volume(landmark_list[4], landmark_list[8])
                    
                    # Brightness (index [8] vs middle [12])
                    update_brightness(landmark_list[8], landmark_list[12])

                # Your existing gesture detection
                detect_gestures(frame, landmark_list, processed)

            frame_counter += 1
            cv2.imshow("Hand Control", frame)

            # Exit on 'q' (corrected waitKey)
            if cv2.waitKey(1) & 0xFF == ord('q'):  # Fix: "waitKey", "0xFF"
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()
if __name__ =="__main__":
    main()

Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 50%
Volume set to 13%
Volume set to 6%
Volume set to 77%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 92%
Volume set to 39%
Volume set to 0%
Volume set to 0%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 100%
Volume set to 42%
Volume set to 11%
Volume set to 0%
Volume set to 69%
Volume set to 82%
Volume set to 80%
Volume set to 81%
Volume set to 80%
Volume set to 79%
Volume set to 85%
Volume set to 84%
Volume set to 84